In [2]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que exista más de un agente por celda, elegimos ''MultiGrid''.
from mesa.space import MultiGrid

# Con ''RandomActivation'', hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random
MAXVAL = 10000

In [3]:
class Basura(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

In [4]:
class Obstaculo(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)

In [5]:
class Papelera(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.position = self.pos

In [6]:
#Nombre: AgenteRobot
#Parametros: Ninguno.
#Return: Nada
#Se encarga de servir como base para crear agentes de tipo aspiradora
class AgenteRobot(Agent):
  def __init__(self, id, model):
    super().__init__(id, model)
    self.totalMoves = 0
    self.target_position = None
    self.current_target = None
    self.target_positions = []
    self.visited_cells = set()
    self.direction = (1, 0)  # Start by moving to the right
    self.actualBoxesCarring = 0
    self.maxBoxesToCarry = 5

  def update_pile_list(self):
    neighbors = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
    for neighbor in neighbors:
      cell_contents = self.model.grid.get_cell_list_contents(neighbor)
      num_boxes = sum(1 for agent in cell_contents if isinstance(agent, Box))
      if 0 < num_boxes < 5:
        if neighbor not in self.target_positions:
          self.target_positions.append(neighbor)
      elif num_boxes >= 5:
        if neighbor in self.target_positions:
          self.target_positions.remove(neighbor)

  def move_towards_target(self):
    current_x, current_y = self.pos
    target_x, target_y = self.target_position
    # Calcular la dirección del movimiento
    dx = target_x - current_x
    dy = target_y - current_y
    # Determinar el paso de movimiento en x y y
    step_x = 1 if dx > 0 else -1 if dx < 0 else 0
    step_y = 1 if dy > 0 else -1 if dy < 0 else 0
    # Crear una lista de posibles movimientos
    possible_moves = []
    if step_x != 0:
      possible_moves.append((current_x + step_x, current_y))
    if step_y != 0:
      possible_moves.append((current_x, current_y + step_y))
        # Filtrar los movimientos posibles para mantener solo los vacíos
    possible_moves = [move for move in possible_moves if self.model.grid.is_cell_empty(move)]
    if possible_moves:
      # Elegir el movimiento más cercano al objetivo
      move_to_target = min(possible_moves, key=lambda pos: self.model.grid.get_distance(pos, self.target_position))
      self.model.grid.move_agent(self, move_to_target)
      self.totalMoves += 1
    else:
      # Si no hay movimientos posibles, quedarse en el mismo lugar
      self.target_position = None

  def move(self):
   if self.current_target and self.carrying_box:
     self.move_towards_target()
   else:
     self.random_move()

  def move_Pond(self):
    neighbors = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
    non_visited_neighbors = [neighbor for neighbor in neighbors if neighbor not in self.visited_cells]
    if non_visited_neighbors:
      new_position = self.random.choice(non_visited_neighbors)
    elif neighbors:
      new_position = self.random.choice(neighbors)

    self.model.grid.move_agent(self, new_position)
    self.visited_cells.add(new_position)
    self.totalMoves += 1


  def move_zigsag(self):
    next_position = (self.pos[0] + self.direction[0], self.pos[1] + self.direction[1])
    if not self.model.grid.out_of_bounds(next_position) and self.model.grid.is_cell_empty(next_position):
      self.model.grid.move_agent(self, next_position)
      self.totalMoves += 1
    else:
      # Change direction at the edges or obstacles
      if self.direction == (1, 0):
        self.direction = (0, 1)  # Move down
      elif self.direction == (0, 1):
        self.direction = (-1, 0)  # Move left
      elif self.direction == (-1, 0):
        self.direction = (0, -1)  # Move up
      elif self.direction == (0, -1):
        self.direction = (1, 0)  # Move right

      next_position = (self.pos[0] + self.direction[0], self.pos[1] + self.direction[1])
      if not self.model.grid.out_of_bounds(next_position) and self.model.grid.is_cell_empty(next_position):
        self.model.grid.move_agent(self, next_position)
        self.totalMoves += 1

  def choose_target(self):
    if self.target_positions:
        self.current_target = self.target_positions.pop(0)  # Elegir el objetivo más cercano
    else:
        self.current_target = None
  def move_Score(self):
    # Obtener las posiciones vecinas
    neighbors = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
    # Calcular el puntaje para cada vecino
    scores = {neighbor: self.calculate_score(neighbor) for neighbor in neighbors}
    # Elegir el vecino con el mayor puntaje
    max_score = max(scores.values())
    best_neighbors = [neighbor for neighbor, score in scores.items() if score == max_score]
    # Si hay múltiples vecinos con el mismo puntaje, elegir uno al azar
    new_position = self.random.choice(best_neighbors)
    # Moverse a la nueva posición
    self.model.grid.move_agent(self, new_position)
    self.totalMoves += 1

  def calculate_score(self, position):
    # Calcular el puntaje basado en la distancia a las celdas vacías y la presencia de cajas
    empty_neighbors = self.model.grid.get_neighborhood(position, moore=False, include_center=False, radius=2)
    empty_score = sum(1 for neighbor in empty_neighbors if self.model.grid.is_cell_empty(neighbor))
    box_neighbors = self.model.grid.get_neighborhood(position, moore=False, include_center=False, radius=1)
    box_score = -sum(1 for neighbor in box_neighbors if not self.model.grid.is_cell_empty(neighbor))
    return empty_score + box_score

  def random_move(self):
    neighbors = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
    empty_neighbors = [neighbor for neighbor in neighbors if self.model.grid.is_cell_empty(neighbor)]
    if empty_neighbors:
      if self.carrying_box and self.target_position:
        if self.pos == self.target_position:
          self.place_box(self.pos)
          self.target_position = None
        else:
          new_position = self.random.choice(empty_neighbors)
          self.model.grid.move_agent(self, new_position)
          self.totalMoves += 1
      else:
        new_position = self.random.choice(empty_neighbors)
        self.model.grid.move_agent(self, new_position)
        self.totalMoves += 1

  def place_box(self, pos):
    new_box = Box(self.model.next_id(), self.model)
    self.model.grid.place_agent(new_box, pos)
    self.carrying_box = False

  def pick_box(self, pos):
    cell_contents = self.model.grid.get_cell_list_contents(pos)
    for agent in cell_contents:
      if isinstance(agent, Box):
        self.carrying_box = True
        self.model.grid.remove_agent(agent)
        return

  def search(self):
    neighbors = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
    # Check for boxes and piles in adjacent cells
    for neighbor in neighbors:
      cell_contents = self.model.grid.get_cell_list_contents(neighbor)
      num_boxes = sum(1 for agent in cell_contents if isinstance(agent, Box))
      if num_boxes > 0:
        if self.carrying_box and num_boxes <= 4:
          self.place_box(neighbor)
          return
        elif not self.carrying_box:
          if num_boxes < 3:
            self.pick_box(neighbor)
            return
        elif num_boxes >= 3:
          self.target_position = neighbor
          return


  def step(self):
    self.update_pile_list()
    self.search()
    self.move()


In [7]:
#Nombre: getGrid
#Parametros: un model.
#Return: Nada
#Se encarga de crear una representación visual del estado actual de la cuadrícula en el modelo
def getGrid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            if not model.grid.is_cell_empty((x, y)):
                contents = model.grid.get_cell_list_contents((x, y))
                if any(isinstance(agent, Basura) for agent in contents):
                    grid[x][y] = 1  # Asignar 1 si hay una caja en la celda
                elif any(isinstance(agent, AgenteRobot) for agent in contents):
                    grid[x][y] = 2  # Asignar 4 si hay un agente robot en la celda
                elif any(isinstance(agent, Papelera) for agent in contents):
                    grid[x][y] = 3  # Asignar 4 si hay un agente robot en la celda
                elif any(isinstance(agent, Papelera) for agent in contents):
                    grid[x][y] = 3  # Asignar 4 si hay un agente robot en la celda
            else:
                grid[x][y] = 0  # Celda vacía
    return grid

In [8]:
class WarehouseModel(Model):
  def __init__(self, width, height, num_agents, num_boxes):
    super().__init__()
    self.cells = np.zeros((width, height))
    self.num_agents = num_agents
    self.num_boxes = num_boxes
    self.grid = MultiGrid(width, height, False)
    self.schedule = RandomActivation(self)
    self.dataCollector = DataCollector(model_reporters = {"Grid" : getGrid })
    self.currentStep = 0


    #Colocamos las cajas
    box_id = 0
    while self.num_boxes > 0:
      num_boxes_in_group = random.randint(1, 3)
      x = self.random.randrange(self.grid.width)
      y = self.random.randrange(self.grid.height)
      # Verificar el número de cajas en la celda
      cell_contents = self.grid.get_cell_list_contents((x, y))
      num_boxes_in_cell = sum(1 for agent in cell_contents if isinstance(agent, Box))
      # Ajustar el grupo de cajas si es mayor que las cajas restantes
      if num_boxes_in_group > self.num_boxes:
        num_boxes_in_group = self.num_boxes
        # Si al agregar el grupo de cajas no se excede el máximo, colocarlas
        if num_boxes_in_cell + num_boxes_in_group <= 5:
          for i in range(num_boxes_in_group):
            new_box = Box(box_id, self)
            self.grid.place_agent(new_box, (x, y))
            box_id += 1
          self.num_boxes -= num_boxes_in_group
        else:
           # Colocar solo el número de cajas que faltan para llegar al máximo de 5
           remaining_space = 5 - num_boxes_in_cell
           for i in range(remaining_space):
            new_box = Box(box_id, self)
            self.grid.place_agent(new_box, (x, y))
            box_id += 1
           self.num_boxes -= remaining_space
      else:
        # Si al agregar el grupo de cajas no se excede el máximo, colocarlas
        if num_boxes_in_cell + num_boxes_in_group <= 5:
          for i in range(num_boxes_in_group):
            new_box = Box(box_id, self)
            self.grid.place_agent(new_box, (x, y))
            box_id += 1
          self.num_boxes -= num_boxes_in_group
        else:
          # Colocar solo el número de cajas que faltan para llegar al máximo de 5
          remaining_space = 5 - num_boxes_in_cell
          for i in range(remaining_space):
            new_box = Box(box_id, self)
            self.grid.place_agent(new_box, (x, y))
            box_id += 1
          self.num_boxes -= remaining_space

      print(f'Cajas restantes: {self.num_boxes}')


    for i in range(self.num_agents):
      empty_positions = self.grid.empties
      if empty_positions:
        position = self.random.choice(list(empty_positions))
        agent = AgenteRobot(i, self)
        self.grid.place_agent(agent, position)
        self.schedule.add(agent)

  def count_piles(self):
    count = 0
    for x in range(self.grid.width):
      for y in range(self.grid.height):
        contents = self.grid.get_cell_list_contents((x, y))
        num_boxes = sum(1 for agent in contents if isinstance(agent, Box))
        if num_boxes == self.max_boxes_per_pile:
          count += 1
    self.pile_count = count

  def all_boxes_stacked(self):
    self.count_piles()
    return self.pile_count == 40

  def count_total_boxes(self):
    total_boxes = 0
    for x in range(self.grid.width):
      for y in range(self.grid.height):
        contents = self.grid.get_cell_list_contents((x, y))
        total_boxes += sum(1 for agent in contents if isinstance(agent, Box))
    return total_boxes

  def step(self):
    self.dataCollector.collect(self)
    self.schedule.step()
    self.count_piles()
    self.currentStep += 1
    #if self.all_boxes_stacked():
     # self.running = False


In [9]:
#Se encarga de llevar acabo las iteraciones, es decir el step, es donde se lleva acabo la simlacion
GRID_SIZE = 20

MAX_ITER = 1000

AGENT_NUM = 5

NUM_BOXES = 200

startTime = time.time()

model = WarehouseModel(GRID_SIZE, GRID_SIZE, AGENT_NUM, NUM_BOXES)

i=1
while i < MAX_ITER:
  model.step()
  i = i + 1

NameError: name 'Box' is not defined

In [ ]:
print('Valores de comprobacion')
model.count_total_boxes()


In [ ]:
#Obtenemos la informacion requerida para el analsis.
all_grid = model.dataCollector.get_model_vars_dataframe()
print(all_grid)

In [ ]:
#Obtenemos la informacion en cada step y la graficamos para despues juntarla en una animacion
fig, axs=plt.subplots(figsize=(4,4))
axs.set_xticks([])
axs.set_yticks([])

if not all_grid.empty:
  print("hell yeah")
  patch = plt.imshow(all_grid.iloc[0][0], cmap='viridis')
else:
  print("no :C")
  patch = plt.imshow(np.zeros((model.grid.width, model.grid.height)), cmap='viridis')

def animate(i):
  if i < len(all_grid):
    patch.set_data(all_grid.iloc[i][0])
  else:
    patch.set_data(all_grid.iloc[-1][0])  # Muestra el último estado disponible

anim = animation.FuncAnimation(fig, animate, frames=MAX_ITER, repeat=False)
plt.show()

In [ ]:
#Llamamos la animacion
anim

In [ ]:
#Despliege de los datos importantes.
totalTime = time.time() - startTime
print('Tiempo de ejecucion : %.5f segundos' % totalTime)
#print(f'Tiempo necesario hasta que todas las celdas estén limpias: {model.timeNeeded} Steps')
print(f'pilas de 5 cajas al término de la simulación: {model.pile_count}')
#print(f'Número de movimientos realizados por todos los agentes: {model.totalMoves}')